# OpenAI Function Calling


A new capability added to the OpenAl API a few months ago.
We'll go over how exactly to use this, and some tips and tricks for getting best results.


OpenAl has fine-tuned the `gpt-3.5-turbo-0613` and `gpt-4-0613` models to:

- Accept additional arguments through which users can pass in descriptions of functions.
- If it is relevant, return the name of the function to use, along with a JSON object with the appropriate input parameters.

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [3]:
# define a function which will then pass to the LLM directly
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location", # Important!
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",  # Important!
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [4]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [6]:
import openai
response = openai.ChatCompletion.create(
    # The latest model which has the function calling capability
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)
print(response)

{
  "id": "chatcmpl-A6IDHXrqtAD4qcncfDZr1gineIIf6",
  "object": "chat.completion",
  "created": 1726063343,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
        },
        "refusal": null
      },
      "logprobs": null,
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 82,
    "completion_tokens": 18,
    "total_tokens": 100
  },
  "system_fingerprint": null
}


In [8]:
response_message = response["choices"][0]["message"]
response_message

<OpenAIObject at 0x7fc4ae5f01d0> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
  },
  "refusal": null
}

In [9]:
response_message["content"] # returns empty

In [10]:
response_message["function_call"]

<OpenAIObject at 0x7fc4ae5860e0> JSON: {
  "name": "get_current_weather",
  "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
}

In [11]:
json.loads(response_message["function_call"]["arguments"])

{'location': 'Boston, MA'}

In [14]:
args = json.loads(response_message["function_call"]["arguments"])

# Function calling with OpenAI still needs to call by youself.Rather, 
# it just tells us what function to call, that's the name, 
# and what the arguments to that function should be.
get_current_weather(args)

'{"location": {"location": "Boston, MA"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

## Test random content with `function_call=auto`

In [17]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",
)
print(response)

{
  "id": "chatcmpl-A6IPTdBmH1usZwM0zxkCo6BawX1GA",
  "object": "chat.completion",
  "created": 1726064099,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?",
        "refusal": null
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 76,
    "completion_tokens": 10,
    "total_tokens": 86
  },
  "system_fingerprint": null
}


This time, in side the `choices`, we don't have the `"function_call": {}` becasue the LLM decide it doesn't need to use a function call

## Test random content with `function_call=none`

In [19]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]## Test the function with random content with `function_call=auto`
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)

{
  "id": "chatcmpl-A6ISFqlnYsEpIHmV4B0oCmdnXIbo0",
  "object": "chat.completion",
  "created": 1726064271,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?",
        "refusal": null
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 77,
    "completion_tokens": 9,
    "total_tokens": 86
  },
  "system_fingerprint": null
}


## Test decent content with `function_call=none`

In [21]:
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)

{
  "id": "chatcmpl-A6ISzSWOtbu2FCIEkCg2X1z4OQtZb",
  "object": "chat.completion",
  "created": 1726064317,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Let me check the current weather in Boston for you.",
        "refusal": null
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 82,
    "completion_tokens": 11,
    "total_tokens": 93
  },
  "system_fingerprint": null
}


It still doesn't call the function because we're forcing it not to call the function.  

## Test random content with `function_call`

The model get confused

In [22]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

{
  "id": "chatcmpl-A6IVQK5UFjiHhHKwPQ9DTT3VCqPwH",
  "object": "chat.completion",
  "created": 1726064468,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"San Francisco, CA\"\n}"
        },
        "refusal": null
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 83,
    "completion_tokens": 12,
    "total_tokens": 95
  },
  "system_fingerprint": null
}


## Force to call a function without provides the 

First, the functions themselves and the descriptions count against the token usage limit that you pass to OpenAl.
If we comment out `functions=functions, function_call={"name": "get_current_weather"},`, we can see that prompt tokens goes down to 15.
In this particular example,
the function definition and the function description
I are taking up a lot of the tokens.
This is important to note because OpenAl models have a token limit on

In [32]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston!",
    }
]

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)

messages.append(response["choices"][0]["message"])

print(response)

{
  "id": "chatcmpl-A6IfbzSaQL9od5Cu8vCHbhOMT7Dj4",
  "object": "chat.completion",
  "created": 1726065099,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n\"location\": \"Boston, MA\"\n}"
        },
        "refusal": null
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 89,
    "completion_tokens": 10,
    "total_tokens": 99
  },
  "system_fingerprint": null
}


In [30]:
args = json.loads(response["choices"][0]["message"]['function_call']['arguments'])

observation = get_current_weather(args)
messages.append(
        {
            # This is used to convey to the language model that it's the repsonse of calling a function
            "role": "function", 
            "name": "get_current_weather",
            "content": observation,
        }
)
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
)
print(response)

{
  "id": "chatcmpl-A6IasXGRHTDpiA6kqLUHdGvn68ZWw",
  "object": "chat.completion",
  "created": 1726064806,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The weather in Boston is currently sunny and windy with a temperature of 72 degrees Fahrenheit.",
        "refusal": null
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 77,
    "completion_tokens": 18,
    "total_tokens": 95
  },
  "system_fingerprint": null
}
